<a href="https://colab.research.google.com/github/Diegoarmando24/Tarea-3/blob/main/Ejercicio_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generación del conjunto de puntos (t_i, y_i) con perturbación aleatoria pequeña
np.random.seed(0)
n = 12
epsilon = 1e-10
t = np.linspace(0, 11, n)                   # 12 puntos equiespaciados de 0 a 11
# Coeficientes exactos de la solución (todos 1)
x_true = np.ones(n)
# Matriz de Vandermonde para polinomio de grado 11
A = np.vander(t, n, increasing=True)
# Valores exactos y (sin ruido) según el polinomio verdadero
y_true = A.dot(x_true)
# Agregar perturbación aleatoria de magnitud epsilon a y
ruido = epsilon * (2*np.random.rand(n) - 1)  # uniforme en [-epsilon, +epsilon]
y = y_true + ruido

# Ajuste del polinomio grado 11 usando Ecuaciones Normales (Cholesky)
M = A.T @ A
c = A.T @ y
L = np.linalg.cholesky(M)
z = np.linalg.solve(L, c)
x_normales = np.linalg.solve(L.T, z)   # coeficientes ajustados por ecuaciones normales

# Ajuste del polinomio usando QR (Householder)
Q, R = np.linalg.qr(A)
x_qr = np.linalg.solve(R, Q.T @ y)     # coeficientes ajustados por QR

# Cálculo de la diferencia con respecto a la solución exacta (x_j = 1 para todo j)
np.set_printoptions(suppress=True, precision=6)
print("Diferencia coef (Normales) =", x_normales - np.ones(n))
print("Diferencia coef (QR)       =", x_qr - np.ones(n))

# Comparación de sensibilidad: norma del error y número de condición de A
err_normales = np.linalg.norm(x_normales - np.ones(n))
err_qr = np.linalg.norm(x_qr - np.ones(n))
cond_A = np.linalg.cond(A)
print("Condición de A =", cond_A)
print("Norma 2 del error de coef (Normales) =", err_normales)
print("Norma 2 del error de coef (QR)       =", err_qr)

# Visualización del ajuste
tt = np.linspace(min(t), max(t), 300)
# Evaluar los polinomios ajustados en una malla fina de t
y_fit_normales = np.polyval(np.flip(x_normales), tt)
y_fit_qr = np.polyval(np.flip(x_qr), tt)
plt.figure()
plt.scatter(t, y, color='red', label="Datos con ruido")
plt.plot(tt, y_fit_normales, label="Ajuste (Normales)")
plt.plot(tt, y_fit_qr, '--', label="Ajuste (QR)")
plt.xlabel("t")
plt.ylabel("y")
plt.title("Ajuste polinomio grado 11 con ruido $\epsilon=10^{-10}$")
plt.legend()
plt.show()
